In [1]:
import psutil
import numpy as np
from data.rosette_temp import Rosette
import pandas as pd
from multiprocess import Pool, get_context
from itertools import product
import os
import pyvista as pv
import random

# Test #1: single rosette

In [ ]:
a = 10
c = 30
f_r0 = random.uniform(0.75, 1.25)
r0 = f_r0 * a
type(r0)

In [2]:
a = 10
c = 30
f_r0 = random.uniform(0.75, 1.25)
# r0 = float(f_r0 * a)
r0 = 8.41
hp = r0
h0 = hp/4
n_arms = 5
print(a, c, r0, h0, hp, n_arms)
print(type(r0))
ros = Rosette(a, c, r0, h0, hp, n_arms)
# ros.unify_mesh()
# pl = ros.plot(op=0.95)
# pl.show()

10 30 8.41 2.1025 8.41 5
<class 'float'>
[6.9137032e+01 6.9137032e+01 1.3066296e+01 1.3066297e+01 6.9137032e+01
 1.3066297e+01 6.9137032e+01 1.3066296e+01 6.9137032e+01 1.3066297e+01
 6.9137032e+01 1.3066297e+01 1.3066297e+01 7.6293944e-08 1.3066296e+01
 1.3066297e+01 1.3066296e+01 1.3066297e+01 1.3066297e+01]


In [ ]:
# calculate volume
v_ros = ros.volume
print(v_ros)

In [ ]:
# calculate minimally bounding sphere
mbs = ros.calc_mbs()
print(mbs)

In [ ]:
# calculate effective density 
rho_eff = v_ros / mbs['v']
print(rho_eff)

In [ ]:
# visualize mbs
pl = ros.plot()
# create sphere with radius r using pyvista
sphere = pv.Sphere(radius=mbs['r'], center=mbs['c'])
# plot sphere with pyvista
pl.add_mesh(sphere, color='red', opacity=0.5)
pl.show()


# Test #2: test m

# Test #2: multiple rosettes, single cpu

# Test #3: multiple rosettes, multiple cpus